<a href="https://colab.research.google.com/github/dlguswls/Study/blob/main/Sparkml_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PySpark Tutorial:**
 https://www.youtube.com/watch?v=_C8kWso4ne4&t=457s

In [42]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Tutorial1 - 맛보기

In [3]:
import pyspark

In [ ]:
import pandas as pd
pd.read_csv('/content/drive/MyDrive/Study/Spark/data/test1.csv')

,Name,age
0,Krish,31
1,Sudhandshu,30
2,Sunny,29


In [ ]:
type(pd.read_csv('/content/drive/MyDrive/Study/Spark/data/test1.csv'))

pandas.core.frame.DataFrame

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [ ]:
spark

로컬에서 실행될 때 항상 하나의 클러스터만 된다! 실제 클라우드 작업 시에는 여러 클러스터와 인스턴스 생성 가능

In [ ]:
df_pyspark = spark.read.csv('/content/drive/MyDrive/Study/Spark/data/test1.csv')

In [ ]:
df_pyspark

DataFrame[_c0: string, _c1: string]

In [ ]:
df_pyspark.show()

+----------+---+
|       _c0|_c1|
+----------+---+
|      Name|age|
|     Krish| 31|
|Sudhandshu| 30|
|     Sunny| 29|
+----------+---+



In [ ]:
# 첫 번째 열이 어떤 값이든 헤더로 간주하기
df_pyspark = spark.read.option('header', 'true').csv('/content/drive/MyDrive/Study/Spark/data/test1.csv')
df_pyspark.show()

+----------+---+
|      Name|age|
+----------+---+
|     Krish| 31|
|Sudhandshu| 30|
|     Sunny| 29|
+----------+---+



In [ ]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_pyspark.head(3)

[Row(Name='Krish', age='31'),
 Row(Name='Sudhandshu', age='30'),
 Row(Name='Sunny', age='29')]

In [ ]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)



## Tutorial2 - PySpark DataFrame_Part1.

**From Now On**
* PySpark DataFrame
* Reading The Dataset
* Checking the Datatypes of the Columns(Schema)
* Selecting Columns And Indexing
* Check Describe option similar to Pandas
* Adding Columns
* Dropping columns
* Renaming Columns

In [ ]:
## 실습용 데이터프레임 만들기
make_dataframe = pd.DataFrame({'Name':['Krish','Sudhanshu','Sunny'],
                               'age':[31, 30, 29],
                               'Experience':[10, 8, 4]})
make_dataframe.to_csv('/content/drive/MyDrive/Study/Spark/data/test1.csv', index = False)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Dataframe').getOrCreate()
spark

Tutorial1에서 Practice 이름으로 하나 만들어서 다른 이름으로 생성하는 건 안되는 듯

In [ ]:
## read the dataset1
df_pyspark = spark.read.option('header','true').csv('/content/drive/MyDrive/Study/Spark/data/test1.csv')
df_pyspark.show()

+---------+---+----------+
|     Name|age|Experience|
+---------+---+----------+
|    Krish| 31|        10|
|Sudhanshu| 30|         8|
|    Sunny| 29|         4|
+---------+---+----------+



In [ ]:
## Check the schema
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)



In [ ]:
# 숫자형도 문자형으로 읽어온다 -> 불러오는 데이터프레임의 모든 기능을 고려하도록 하는 옵션 추가
## inferSchema = True
df_pyspark = spark.read.option('header','true').csv('/content/drive/MyDrive/Study/Spark/data/test1.csv', inferSchema = True)
df_pyspark.show()

+---------+---+----------+
|     Name|age|Experience|
+---------+---+----------+
|    Krish| 31|        10|
|Sudhanshu| 30|         8|
|    Sunny| 29|         4|
+---------+---+----------+



In [ ]:
df_pyspark.printSchema()
# 숫자형을 integer로 인식!

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [ ]:
## read the dataset2 -- 이 방법이 나은 듯
df_pyspark = spark.read.csv('/content/drive/MyDrive/Study/Spark/data/test1.csv', header = True, inferSchema = True)
df_pyspark.show()

+---------+---+----------+
|     Name|age|Experience|
+---------+---+----------+
|    Krish| 31|        10|
|Sudhanshu| 30|         8|
|    Sunny| 29|         4|
+---------+---+----------+



In [ ]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [ ]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [ ]:
## Selecting Columns And Indexing
df_pyspark.columns

['Name', 'age', 'Experience']

In [ ]:
# 행 인덱싱1
df_pyspark.head(3)

[Row(Name='Krish', age=31, Experience=10),
 Row(Name='Sudhanshu', age=30, Experience=8),
 Row(Name='Sunny', age=29, Experience=4)]

In [ ]:
# 열 인덱싱1
df_pyspark.select('Name').show()

+---------+
|     Name|
+---------+
|    Krish|
|Sudhanshu|
|    Sunny|
+---------+



In [ ]:
type(df_pyspark.select('Name'))

pyspark.sql.dataframe.DataFrame

In [ ]:
# 열 인덱싱2 - 여러 개의 열 선택하기
df_pyspark.select(['Name','Experience']).show()

+---------+----------+
|     Name|Experience|
+---------+----------+
|    Krish|        10|
|Sudhanshu|         8|
|    Sunny|         4|
+---------+----------+



In [ ]:
df_pyspark['Name']  # 이건 .show()로 데이터셋을 가져올 수가 없다

Column<'Name'>

In [ ]:
df_pyspark.dtypes

[('Name', 'string'), ('age', 'int'), ('Experience', 'int')]

In [ ]:
## Check Describe option similar to Pandas
df_pyspark.describe().show()
# 문자열의 최대 최소는 인덱스 값 기준으로 최대, 최소 인덱스에 해당하는 값이 출력됨

+-------+-----+----+-----------------+
|summary| Name| age|       Experience|
+-------+-----+----+-----------------+
|  count|    3|   3|                3|
|   mean| null|30.0|7.333333333333333|
| stddev| null| 1.0|3.055050463303893|
|    min|Krish|  29|                4|
|    max|Sunny|  31|               10|
+-------+-----+----+-----------------+



In [ ]:
## Adding & Dropping Columns

# 새로운 열 추가하거나 기존 열 교체
df_pyspark = df_pyspark.withColumn('Experience After 2 year', df_pyspark['Experience']+2)
df_pyspark.show()

+---------+---+----------+-----------------------+
|     Name|age|Experience|Experience After 2 year|
+---------+---+----------+-----------------------+
|    Krish| 31|        10|                     12|
|Sudhanshu| 30|         8|                     10|
|    Sunny| 29|         4|                      6|
+---------+---+----------+-----------------------+



In [ ]:
# 열 삭제하기
df_pyspark = df_pyspark.drop('Experience After 2 year')
df_pyspark.show()

+---------+---+----------+
|     Name|age|Experience|
+---------+---+----------+
|    Krish| 31|        10|
|Sudhanshu| 30|         8|
|    Sunny| 29|         4|
+---------+---+----------+



In [ ]:
## Renaming Columns
df_pyspark.withColumnRenamed('Name','New Name').show()

+---------+---+----------+
| New Name|age|Experience|
+---------+---+----------+
|    Krish| 31|        10|
|Sudhanshu| 30|         8|
|    Sunny| 29|         4|
+---------+---+----------+



## Tutorial3 - Pyspark Dataframe_Handling Missing Values

**From Now On**
* Dropping Columns
* Dropping Rows
* Various Parameter in Dropping functionalities
* Handling Missing vlaues by Mean, Median, Mode

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practice').getOrCreate()
spark

In [ ]:
## 실습용 데이터프레임 만들기
import numpy as np
make_dataframe = pd.DataFrame({'Name':['Krish','Sudhanshu','Sunny','Paul','Harsha','Shubbam','Mahesh', np.NaN,np.NaN],
                               'age':[31, 30, 29,24, 21, 23, np.NaN, 34, 36],
                               'Experience':[10, 8, 4, 3, 1, 2, np.NaN, 10, np.NaN],
                               'Salary': [30000,25000,20000,20000,15000,18000,40000,38000,np.NaN]})
make_dataframe.to_csv('/content/drive/MyDrive/Study/Spark/data/test2.csv', index = False)

In [ ]:
df_pyspark = spark.read.csv('/content/drive/MyDrive/Study/Spark/data/test2.csv',header = True, inferSchema = True)
df_pyspark.show()

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubbam|23.0|       2.0|18000.0|
|   Mahesh|null|      null|40000.0|
|     null|34.0|      10.0|38000.0|
|     null|36.0|      null|   null|
+---------+----+----------+-------+



In [ ]:
## Dropping Columns
df_pyspark.drop('Name').show()

+----+----------+-------+
| age|Experience| Salary|
+----+----------+-------+
|31.0|      10.0|30000.0|
|30.0|       8.0|25000.0|
|29.0|       4.0|20000.0|
|24.0|       3.0|20000.0|
|21.0|       1.0|15000.0|
|23.0|       2.0|18000.0|
|null|      null|40000.0|
|34.0|      10.0|38000.0|
|36.0|      null|   null|
+----+----------+-------+



In [ ]:
## Dropping Rows
df_pyspark.na.drop().show()  # null 값이 존재하는 모든 행 삭제

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubbam|23.0|       2.0|18000.0|
+---------+----+----------+-------+



In [ ]:
df_pyspark.na.drop(how = 'all').show()  
# any : 하나라도 null값이 있으면 삭제
# all : 모든 값이 null이면 삭제

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubbam|23.0|       2.0|18000.0|
|   Mahesh|null|      null|40000.0|
|     null|34.0|      10.0|38000.0|
|     null|36.0|      null|   null|
+---------+----+----------+-------+



In [ ]:
# threshold(임계값) 설정
df_pyspark.na.drop(how = 'any', thresh = 2).show()  
# any, thresh = 2 : null이 아닌 값이 최소한 두 개 있어야 한다 -> 마지막 줄만 삭제

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubbam|23.0|       2.0|18000.0|
|   Mahesh|null|      null|40000.0|
|     null|34.0|      10.0|38000.0|
+---------+----+----------+-------+



In [ ]:
# subset
df_pyspark.na.drop(how = 'any', subset = ['Experience']).show()    
# Experience 열에 null이 있는 행 삭제

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubbam|23.0|       2.0|18000.0|
|     null|34.0|      10.0|38000.0|
+---------+----+----------+-------+



In [ ]:
## Filing the Missing Value
# null 값을 대체할 값의 타입과 동일한 타입을 가진 열의 null값만 대체된다! 
df_pyspark.na.fill('Missing Values').show()

+--------------+----+----------+-------+
|          Name| age|Experience| Salary|
+--------------+----+----------+-------+
|         Krish|31.0|      10.0|30000.0|
|     Sudhanshu|30.0|       8.0|25000.0|
|         Sunny|29.0|       4.0|20000.0|
|          Paul|24.0|       3.0|20000.0|
|        Harsha|21.0|       1.0|15000.0|
|       Shubbam|23.0|       2.0|18000.0|
|        Mahesh|null|      null|40000.0|
|Missing Values|34.0|      10.0|38000.0|
|Missing Values|36.0|      null|   null|
+--------------+----+----------+-------+



In [ ]:
df_pyspark.na.fill(0, ['Experience','age']).show() # integer type이니까 integer값으로 대체 값 설정

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubbam|23.0|       2.0|18000.0|
|   Mahesh| 0.0|       0.0|40000.0|
|     null|34.0|      10.0|38000.0|
|     null|36.0|       0.0|   null|
+---------+----+----------+-------+



In [ ]:
df_pyspark.na.fill({'Experience': 50,'age': 50,'Name':'unknown'}).show()   # 대체할 null 값을 컬럼별로 지정하기

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubbam|23.0|       2.0|18000.0|
|   Mahesh|50.0|      50.0|40000.0|
|  unknown|34.0|      10.0|38000.0|
|  unknown|36.0|      50.0|   null|
+---------+----+----------+-------+



In [ ]:
## Handling Missing vlaues by Mean, Median, Mode
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['age','Experience','Salary'],
    outputCols = ['{}_imputed'.format(c) for c in ['age','Experience','Salary']]).setStrategy("mean")
# setStrategy값을 median, mode 등으로 바꾸는것에 따라 null값이 대체되는 값이 달라짐

In [ ]:
imputer

Imputer_c8c6dff7994a

In [ ]:
# Add imputation cols go df
imputer.fit(df_pyspark).transform(df_pyspark).show()  
# null 값이 mean값으로 대체된 결과가 _imputed 열에 출력

+---------+----+----------+-------+-----------+------------------+--------------+
|     Name| age|Experience| Salary|age_imputed|Experience_imputed|Salary_imputed|
+---------+----+----------+-------+-----------+------------------+--------------+
|    Krish|31.0|      10.0|30000.0|       31.0|              10.0|       30000.0|
|Sudhanshu|30.0|       8.0|25000.0|       30.0|               8.0|       25000.0|
|    Sunny|29.0|       4.0|20000.0|       29.0|               4.0|       20000.0|
|     Paul|24.0|       3.0|20000.0|       24.0|               3.0|       20000.0|
|   Harsha|21.0|       1.0|15000.0|       21.0|               1.0|       15000.0|
|  Shubbam|23.0|       2.0|18000.0|       23.0|               2.0|       18000.0|
|   Mahesh|null|      null|40000.0|       28.5| 5.428571428571429|       40000.0|
|     null|34.0|      10.0|38000.0|       34.0|              10.0|       38000.0|
|     null|36.0|      null|   null|       36.0| 5.428571428571429|       25750.0|
+---------+----+

## Tutorial4 - Pyspark Dataframes_Filter operation

**From Now On**
* Filter Operation
* &, |, ==
* ~

In [ ]:
## 실습용 데이터프레임 만들기
import numpy as np
make_dataframe = pd.DataFrame({'Name':['Krish','Sudhanshu','Sunny','Paul','Harsha','Shubbam'],
                               'age':[31, 30, 29, 24, 21, 23],
                               'Experience':[10, 8, 4, 3, 1, 2],
                               'Salary': [30000,25000,20000,20000,15000,18000]})
make_dataframe.to_csv('/content/drive/MyDrive/Study/Spark/data/test1.csv', index = False)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practice').getOrCreate()
spark

In [ ]:
df_pyspark = spark.read.csv('/content/drive/MyDrive/Study/Spark/data/test1.csv',header = True, inferSchema = True)
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubbam| 23|         2| 18000|
+---------+---+----------+------+



Filter Operation

In [ ]:
### Salary of the people less than or equal to 20000
df_pyspark.filter("Salary<=20000").show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubbam| 23|         2| 18000|
+-------+---+----------+------+



In [ ]:
df_pyspark.filter("Salary<=20000").select(['Name','age']).show()

+-------+---+
|   Name|age|
+-------+---+
|  Sunny| 29|
|   Paul| 24|
| Harsha| 21|
|Shubbam| 23|
+-------+---+



In [ ]:
df_pyspark.filter(df_pyspark['Salary']<=20000).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubbam| 23|         2| 18000|
+-------+---+----------+------+



In [ ]:
df_pyspark.filter((df_pyspark['Salary']<=20000) & (df_pyspark['Salary']>=15000)).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubbam| 23|         2| 18000|
+-------+---+----------+------+



In [ ]:
df_pyspark.filter((df_pyspark['Salary']<=20000) | (df_pyspark['Salary']>=15000)).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubbam| 23|         2| 18000|
+---------+---+----------+------+



In [ ]:
df_pyspark.filter(~(df_pyspark['Salary']<=20000)).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
+---------+---+----------+------+



## Tutorial5 - Pyspart with Python_GroupBy And Aggregate Functions

In [4]:
## 실습용 데이터프레임 만들기
import pandas as pd

make_dataframe = pd.DataFrame({'Name':['Krish','Krish', 'Mahesh', 'Krish', 'Mahesh','Sudhanshu','Sudhanshu', 'Sudhanshu', 'Sunny', 'Sunny'],
                               'Departments':['Data Science','IOT','Big Data','Big Data','Data Science', 'Data Science', 'IOT','Big Data','Data Science','Big Data'],
                               'Salary': [10000, 5000, 4000, 4000, 3000, 20000, 10000, 5000, 10000, 2000]})
make_dataframe.to_csv('/content/drive/MyDrive/Study/Spark/data/test3.csv', index = False)

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practice').getOrCreate()
spark

In [6]:
df_pyspark = spark.read.csv('/content/drive/MyDrive/Study/Spark/data/test3.csv',header = True, inferSchema = True)
df_pyspark.show()

+---------+------------+------+
|     Name| Departments|Salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [7]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
## Groupby

# Grouped to find the maximum salary
df_pyspark.groupBy('Name').sum().show()

+---------+-----------+
|     Name|sum(Salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



In [16]:
df_pyspark.groupBy('Name').min().show()

+---------+-----------+
|     Name|min(Salary)|
+---------+-----------+
|Sudhanshu|       5000|
|    Sunny|       2000|
|    Krish|       4000|
|   Mahesh|       3000|
+---------+-----------+



In [17]:
df_pyspark.groupBy('Name').avg().show()

+---------+------------------+
|     Name|       avg(Salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



In [12]:
# Groupby Departments which gives maximum salary
df_pyspark.groupBy('Departments').sum().show()

+------------+-----------+
| Departments|sum(Salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



In [13]:
df_pyspark.groupBy('Departments').mean().show()

+------------+-----------+
| Departments|avg(Salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



In [14]:
df_pyspark.groupBy('Departments').count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



In [18]:
df_pyspark.agg({'Salary':'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+



## Tutorial6 - Example Of Pyspark ML

https://spark.apache.org/examples.html

In [19]:
## 실습용 데이터프레임 만들기
import pandas as pd

make_dataframe = pd.DataFrame({'Name':['Krish','Sudhanshu', 'Sunny', 'Paul', 'Harsha','Shubham'],
                               'age':[31, 30, 29, 24, 21, 23],
                               'Experience':[10, 8, 4, 3, 1, 2],
                               'Salary': [30000, 25000, 20000, 20000, 15000, 18000]})
make_dataframe.to_csv('/content/drive/MyDrive/Study/Spark/data/test1.csv', index = False)

In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practice').getOrCreate()
spark

In [22]:
training = spark.read.csv('/content/drive/MyDrive/Study/Spark/data/test1.csv',header = True, inferSchema = True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [23]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [24]:
training.columns

['Name', 'age', 'Experience', 'Salary']

['Age', 'Experience'] --> new feature --> independent feature

In [25]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['age','Experience'], outputCol = 'Independent Features')

In [26]:
output = featureassembler.transform(training)

In [27]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [29]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [30]:
finalized_data = output.select("Independent Features",'Salary')

In [31]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [34]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = "Independent Features", labelCol = 'Salary')
regressor = regressor.fit(train_data)

In [35]:
# Coefficients
regressor.coefficients

DenseVector([555.5556, 714.2857])

In [36]:
regressor.intercept  # 절편

2619.047619047663

In [37]:
pred_results = regressor.evaluate(test_data)

In [38]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000| 16825.39682539683|
|          [24.0,3.0]| 20000|  18095.2380952381|
|          [29.0,4.0]| 20000|21587.301587301583|
|         [31.0,10.0]| 30000|26984.126984126975|
+--------------------+------+------------------+



In [40]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1920.6349206349196, 4155706.727135302)

# **PySpark Tutorial: Spark SQL & DataFrame Basics**
https://www.youtube.com/watch?v=3-pnWVWyH-s

In [43]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [44]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [45]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [47]:
# Load the csv into a dataframe
titanic_df = spark.read.csv("/content/drive/MyDrive/Study/Spark/data/train.csv", header=True, inferSchema=True)
titanic_df

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,null,S
6,0,3,"Moran, Mr. James",male,null,0,0,330877,8.4583,null,Q
7,0,1,"McCarthy, Mr. Tim...",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. ...",male,2.0,3,1,349909,21.075,null,S
9,1,3,"Johnson, Mrs. Osc...",female,27.0,0,2,347742,11.1333,null,S
10,1,2,"Nasser, Mrs. Nich...",female,14.0,1,0,237736,30.0708,null,C


In [50]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [52]:
titanic_df.limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,null,S


In [53]:
titanic_df.select('PassengerId', 'Survived').limit(5)

PassengerId,Survived
1,0
2,1
3,1
4,1
5,0


In [54]:
titanic_df.where((titanic_df.Age > 25) & (titanic_df.Survived == 1)).limit(5)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
9,1,3,"Johnson, Mrs. Osc...",female,27.0,0,2,347742,11.1333,null,S
12,1,1,"Bonnell, Miss. El...",female,58.0,0,0,113783,26.55,C103,S


In [55]:
titanic_df.agg({'Fare':'avg'})

avg(Fare)
32.2042079685746


In [56]:
titanic_df.groupBy('Pclass').agg({'Fare':'avg'}).orderBy('Pclass', ascending=False)

Pclass,avg(Fare)
3,13.675550101832997
2,20.66218315217391
1,84.15468749999992


In [57]:
titanic_df.filter(titanic_df.Age > 25).agg({'Fare':'avg'})

avg(Fare)
37.61960169491524


In [59]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def round_float_down(x):
  return int(x)

round_float_down_udf = udf(round_float_down, IntegerType())

titanic_df.select('PassengerId', 'Fare', round_float_down_udf('Fare').alias('Fare Rounded Down'))

PassengerId,Fare,Fare Rounded Down
1,7.25,7
2,71.2833,71
3,7.925,7
4,53.1,53
5,8.05,8
6,8.4583,8
7,51.8625,51
8,21.075,21
9,11.1333,11
10,30.0708,30


In [60]:
titanic_df.createOrReplaceTempView("Titanic")

In [61]:
spark.sql('select * from Titanic')

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,null,S
6,0,3,"Moran, Mr. James",male,null,0,0,330877,8.4583,null,Q
7,0,1,"McCarthy, Mr. Tim...",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. ...",male,2.0,3,1,349909,21.075,null,S
9,1,3,"Johnson, Mrs. Osc...",female,27.0,0,2,347742,11.1333,null,S
10,1,2,"Nasser, Mrs. Nich...",female,14.0,1,0,237736,30.0708,null,C
